In [2]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import os
os.chdir("/Users/mammoc/Desktop/Python")
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
airport = pd.read_csv("airport_original1.csv", sep = ';', header = 0)
#airport

,ID,Airport,Airline,Age,Gender,Nationality,TripPurpose,TripDuration,FlyingCompanion,ProvinceResidence,...,SeatClass,Airfare,NoTransport,ModeTransport,AccessCost,AccessTime,Occupation,Income,MileageAirline,Mileage
0,1,1,1.0,49.0,1.0,1,2,7,0,3,...,1.0,80.0,1,6,8000.0,40.0,1,5.0,1,150000.0
1,2,1,1.0,49.0,2.0,1,1,4,4,3,...,1.0,41.0,1,6,8000.0,50.0,9,3.0,NaN,NaN
2,3,1,1.0,25.0,1.0,1,1,10,2,3,...,1.0,NaN,2,4,1000.0,20.0,12,NaN,NaN,NaN
3,4,1,1.0,29.0,1.0,1,2,7,2,3,...,1.0,40.0,1,1,NaN,NaN,8,7.0,2,100000.0
4,5,1,1.0,34.0,2.0,1,2,4,0,3,...,1.0,40.0,1,6,8000.0,50.0,1,3.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,484,2,4.0,39.0,1.0,1,4,8,3,8,...,1.0,62.0,1,1,NaN,45.0,12,1.0,5,1.0
484,485,2,4.0,37.0,2.0,1,4,8,3,8,...,1.0,25.0,1,1,NaN,45.0,5,5.0,NaN,NaN
485,486,2,1.0,63.0,1.0,5,2,28,1,8,...,1.0,NaN,2,5,NaN,30.0,2,7.0,3,NaN
486,487,2,4.0,61.0,1.0,3,2,2,4,8,...,2.0,17.0,1,2,30000.0,40.0,1,7.0,2,30000.0


In [3]:
# Dropping rows where 'Destination' is 4
airport = airport[airport['Destination'] != 4]
airport = airport.dropna(subset=['Destination'])

In [4]:
#FILL UP MISSING VALUES
#Age has 1 missing value (ID 214) => Replace by average age of 40
airport.loc[airport['ID'] == 214, 'Age'] = 40

#Gender has 3 missing values (ID 179, 199, 259)
# => Fill up by most frequency category - female - 2.0
airport.loc[airport['ID'].isin([179, 199, 259]), 'Gender'] = 2.0


#SeatClass has 3 missing value 
#Fill up by most frequency category = 1.0
airport.loc[airport['ID'].isin([245, 255, 450, 475]), 'SeatClass'] = 1.0

#AccessTime
AccessTime_by_ProvinceResidence = airport.groupby('ProvinceResidence')['AccessTime'].mean()
airport['AccessTime'] = airport.apply(
    lambda row: AccessTime_by_ProvinceResidence[row['ProvinceResidence']] if pd.isnull(row['AccessTime']) 
    else row['AccessTime'],axis=1)
#airport.loc[airport['ID'] == 102, 'AccessTime']
#AccessTime_by_ProvinceResidence


# Recategorize 'Airline' into binary categories: 'Legacy' and 'Other'
airport['Airline[Legacy]'] = airport['Airline'].isin([1, 2]).astype(int)

Airfare_by_Airline_Destination = airport.groupby(['Airline[Legacy]','Destination'])['Airfare'].mean()

#Fill the missing values in 'Airfare' with the average values calculated above
airport['Airfare'] = airport.apply(
    lambda row: Airfare_by_Airline_Destination[row['Airline[Legacy]'], row['Destination']] if pd.isnull(row['Airfare']) 
    else row['Airfare'], axis=1)


In [7]:
X = airport[['Age',
             'Gender',
             'TripPurpose',
             'Nationality',
             'TripDuration',
             'FlyingCompanion',
             'ProvinceResidence',
             'GroupTravel',
             'Destination',
             'DepartureTime',            
             'AccessTime',
             'NoTransport',
             'NoTripsLastYear',
             'SeatClass',
             'ModeTransport',
             'Airfare'
             ]]
#Seoul,Incheon and other, Incheon,Kyungki-do and others
y = airport['Airport']
airport['Airline[Legacy]'] = (airport['Airline'].isin([1,2])).astype(int)
y2 = airport['Airline[Legacy]']
random_state=109

In [8]:
#Standardize
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

In [17]:
svclassifier = SVC(kernel='rbf') #rbf = 81
svclassifier.fit(X, y)

SVC()

In [18]:
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

# Assuming X and y are your features and target variable, respectively
svclassifier = SVC(kernel='rbf')

# Perform 10-fold cross-validation
cv_scores = cross_val_score(svclassifier, X, y, cv=10)

# cv_scores will contain the accuracy scores for each fold
print("Accuracy scores for each fold:", cv_scores)

# To get an overall idea of the model's performance, you might want to look at the mean accuracy across all folds
print("Mean cross-validation accuracy:", cv_scores.mean())

# It's also useful to see the standard deviation to understand the variability between folds
print("Standard deviation of cross-validation accuracy:", cv_scores.std())


Accuracy scores for each fold: [0.76086957 0.63043478 0.7173913  0.80434783 0.93478261 0.80434783
 0.84782609 0.91304348 0.7826087  0.93333333]
Mean cross-validation accuracy: 0.8128985507246377
Standard deviation of cross-validation accuracy: 0.09316360929589641


In [25]:
svclassifier = SVC(kernel='linear') #rbf = 65, linear = 69
svclassifier.fit(X, y2)

SVC(kernel='linear')

In [30]:
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

# Assuming X and y are your features and target variable, respectively
svclassifier = SVC(kernel='linear')

# Perform 10-fold cross-validation
cv_scores = cross_val_score(svclassifier, X, y2, cv=10)

# cv_scores will contain the accuracy scores for each fold
print("Accuracy scores for each fold:", cv_scores)

# To get an overall idea of the model's performance, you might want to look at the mean accuracy across all folds
print("Mean cross-validation accuracy:", cv_scores.mean())

# It's also useful to see the standard deviation to understand the variability between folds
print("Standard deviation of cross-validation accuracy:", cv_scores.std())


Accuracy scores for each fold: [0.7173913  0.80434783 0.73913043 0.63043478 0.60869565 0.63043478
 0.67391304 0.67391304 0.7826087  0.68888889]
Mean cross-validation accuracy: 0.6949758454106281
Standard deviation of cross-validation accuracy: 0.06228445293237025
